# Missing Data

Missing data is a common problem in most real-world scientific datasets. While the best way for dealing with missing data will always be preventing their occurrence in the first place, it usually can't be helped, particularly when data are collected passively or voluntarily, or when data collection and recording is distributed among several people. There are a variety of ways for dealing with missing data, from the very naïve to the very sophisticated, and unfortunately the more common approaches tend to be *ad hoc* and will usually do more harm than good. 

It turns out that more robust methods for imputation are not as difficult to implement as they first appear to be. ADD REFERENCES Two of the best ones are Bayesian imputation and multiple imputation. In this section, we will use multiple imputation to account for missing data in a regression analysis. 

As a motivating example, we will use a dataset of educational outcomes for children with hearing impairment. Here, we are interested in determining factors that are associated with better or poorer learning outcomes. There is a suite of available predictors, including gender (`male`), number of siblings in the household (`siblings`), an index of family involvement (`family_inv`), whether the primary household language is not English (`non_english`), the presence of a previous disability (`prev_disab`), non-white race (`non_white`), the age at the time of testing (in months, `age_test`), whether hearing loss is not severe (`non_severe_hl`), whether the subject's mother obtained a high school diploma or better (`mother_hs`), and whether the hearing impairment was identified by 3 months of age (`early_ident`).

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
test_scores = pd.read_csv('../data/test_scores.csv', index_col=0)
test_scores.head()

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2,2,False,NaN,55,1,NaN,False,False
1,31,1,0,NaN,False,0,53,0,0,False,False
2,83,1,1,1,True,0,52,1,NaN,False,True
3,75,0,3,NaN,False,0,55,0,1,False,False
5,62,0,0,4,False,1,50,0,NaN,False,False


For three variables in the dataset, there are incomplete records.

In [37]:
test_scores.isnull().sum(0)

score             0
male              0
siblings          0
family_inv       33
non_english       0
prev_disab       18
age_test          0
non_severe_hl     0
mother_hs        73
early_ident       0
non_white         0
dtype: int64

## Strategies for dealing with missing data

The easiest (and worst) way to deal with missing data is to ignore it. That is, simply run the analysis, missing values and all, hoping for the best. If your software is any good, this approach will simply not work; the algorithm will try to operate on data that includes missing values, and propagate them, resulting in statistics and estimates that cannot be calculated, which will typically raise errors. If your software is poor, it will make some assumption or decision about the missing values, and proceed to generate  results conditional on the assumption, which creates problems that may never be detected because no indication was given to any potential problem. 

The next easiest (worst) approach to analyzing data with missing values is to conduct list-wise deletion, by deleting the records that have missing values. This is called **complete case analysis**, because only records that are complete get retained for the analysis. The degree to which complete case analysis is undesirable depends on the mechanism by which data have become missing.

## Types of Missingness

- **Missing completely at random (MCAR)**: When data are MCAR, missing cases are, on average, identical to non-missing cases, with respect to the model. Ignoring the missingness will reduce the power of the analysis, but will not bias inference.
- **Missing at random (MAR)**: Missing data depends (usually probabilistically) on measured values, and hence can be modeled by variables observed in the data set. Accounting for the values which “cause” the missing data will produce unbiased results in an analysis.
- **Missing not at random(MNAR)**: Missing data depend on unmeasured or unknown variables. There is no information available to account for the missingness.

The very best-case scenario for using complete case analysis, which corresponds to MCAR missingness, results in a loss of power due to the reduction in sample size. The analysis will lose the information contained in the non-missing elements of a partially-missing record. When data are not missing completely at random, inferences from complete case analysis may be biased due to systematic differences between missing and non-missing records that affects the estimates of key parameters.

One alternative to complete case analysis is to simply fill (*impute*) the missing values with a reasonable guess a the true value, such as the mean, median or modal value of the fully-observed records. This imputation, while not recovering any information regarding the missing value itself for use in the analysis, does provide a mechanism for including the non-missing values in the analysis, thereby making use of all available information.

Performing imputation via mean substitution is easy in Pandas, via the DataFrame/Series `fillna` method.

In [38]:
test_scores.siblings.mean()

1.1256038647342994

In [39]:
test_scores.siblings.fillna(test_scores.siblings.mean())

0      2
1      0
2      1
3      3
5      0
6      1
7      2
9      0
10     0
12     1
13     0
14     2
16     2
17     1
18     1
19     0
21     1
23     1
25     1
26     1
27     1
28     1
29     1
30     2
31     1
32     1
33     2
34     1
35     0
36     1
      ..
194    1
195    1
196    0
198    1
199    1
200    1
201    1
202    3
203    1
204    1
205    0
206    3
207    1
208    1
209    0
210    0
211    0
212    1
213    1
214    2
215    2
216    1
217    1
218    2
219    1
220    2
221    2
222    1
223    2
224    2
Name: siblings, dtype: float64

This approach may be reasonable under the MCAR assumption, but may induce bias under a MAR scenario, whereby missing values may differ systematically relative to non-missing values, making the particular summary statistic used for imputation *biased* as a mean/median/modal value for the missing values.

Beyond this, the use of a single imputed value to stand in place of the actual missing value glosses over the *uncertainty* associated with this guess at the true value. Any subsequent analysis procedure (*e.g.* regression analysis) will behave as if the imputed value were observed, despite the fact that we are actually unsure of the actual value for the missing variable. The practical consequence of this is that the variance of any estimates resulting from the imputed dataset will be artificially reduced.

## Multiple Imputation

One robust alternative to addressing missing data is **multiple imputation** (Schaffer 1999, van Buuren 2012). It produces unbiases parameter estimates, while simultaneously accounting for the uncertainty associated with imputing missing values. It is conceptually and mechanistically straightforward, and produces complete datasets that may be analyzed using any statistical methodology or software one chooses, as if the data had no missing values to begin with.

Multiple imputation generates imputed values based on a regression model. This regression model will help us generate reasonable values, particularly if data are MAR, since it uses information in the dataset that may be informative in predicting what the true value may be. Ideally, we want predictor variables that are correlated with the missing variable, and with the mechanism of missingness, if any. For example, one might be able to use test scores from one subject to predict missing test scores from another; or, the probability of income reporting to be missing may vary systematically according to the education level of the individual.

To see if there is any potential information among the variables in our dataset to use for imputation, it is helpful to calculate the pairwise correlation between all the variables. Since we have discrete variables in our data, the [Spearman rank correlation coefficient](http://www.wikiwand.com/en/Spearman%27s_rank_correlation_coefficient) is appropriate.

In [40]:
test_scores.dropna().corr(method='spearman')

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
score,1.000000,0.073063,-0.085044,-0.539019,-0.278798,-0.184426,0.024057,0.140305,0.228500,0.222711,-0.345061
male,0.073063,1.000000,-0.072006,-0.008714,0.053338,-0.052054,-0.081165,0.031825,0.050372,-0.007690,-0.048638
siblings,-0.085044,-0.072006,1.000000,0.078471,-0.049989,-0.038020,0.104905,-0.003689,0.096268,0.077318,0.006234
family_inv,-0.539019,-0.008714,0.078471,1.000000,0.221696,0.082314,-0.029120,-0.092815,-0.358898,0.006370,0.401617
non_english,-0.278798,0.053338,-0.049989,0.221696,1.000000,-0.021996,0.068095,-0.047775,-0.199639,-0.015812,0.225428
prev_disab,-0.184426,-0.052054,-0.038020,0.082314,-0.021996,1.000000,0.136604,0.048132,0.137893,0.046592,-0.021367
age_test,0.024057,-0.081165,0.104905,-0.029120,0.068095,0.136604,1.000000,-0.122811,0.016760,0.033789,0.068430
non_severe_hl,0.140305,0.031825,-0.003689,-0.092815,-0.047775,0.048132,-0.122811,1.000000,-0.015996,0.008211,0.028480
mother_hs,0.228500,0.050372,0.096268,-0.358898,-0.199639,0.137893,0.016760,-0.015996,1.000000,0.024411,-0.214209
early_ident,0.222711,-0.007690,0.077318,0.006370,-0.015812,0.046592,0.033789,0.008211,0.024411,1.000000,-0.022854


We will try to impute missing values the mother's high school education indicator variable, which takes values of 0 for no high school diploma, or 1 for high school diploma or greater. The appropriate model to predict binary variables is a **logistic regression**. We will use the scikit-learn implementation, `LogisticRegression`.

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

To keep things simple, we will only use variables that are themselves complete to build the predictive model, hence our subset of predictors will exclude family involvement score (`family_inv`) and previous disability (`prev_disab`).

In [42]:
impute_subset = test_scores.drop(labels=['family_inv','prev_disab','score'], axis=1)

Next, we scale the predictor variables to range from 0 to 1, to improve the performance of the regression model.

In [43]:
y = impute_subset.pop('mother_hs').values
X = StandardScaler().fit_transform(impute_subset.astype(float))

The *training* and *test* sets in this case will be the non-missing and missing values, respectively, since we want to use supervised learning to build our predictive model.

In [44]:
missing = np.isnan(y)

Next, we create a `LogisticRegression` model, and fit it using the non-missing observations.

In [45]:
mod = LogisticRegression()
mod.fit(X[~missing], y[~missing])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

Conveniently, scikit-learn models have `predict` methods for generating predictions from the model, using new data. Here, we will pass the predictor values for the subset with `mother_hs` missing.

In [46]:
mother_hs_pred = mod.predict(X[missing])
mother_hs_pred

array([ 1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.])

These values can then be inserted in place of the missing values, and an analysis can be performed on the entire dataset.

However, this is still just a single imputation for each missing value, and hence glosses over the uncertainty associated with the derivation of the imputes. Multiple imputation proceeds by imputing several values, to generate several complete datasets and performing the same analysis on all of them. With a set of estimates in hand, an *average* estimate can be obtained that more adequately accounts for the uncertainty, hopefully providing more robust inference than from a single impute.

There are a variety of ways to generate multiple imputations. We will exploit regularization in order to do this. The `LogisticRegression` class from scikit-learn provides facilities for regularization using either L2 (resulting in ridge regression) or L1 (resulting in LASSO regression) penalties. The degree of regularization in either case is controlled by the `C` parameter, whereby large values of `C` give more freedom to the model, while smaller values of C constrain the model more. We can use a selection of `C` values to obtain a range of predictions from variants of the same model. For example:

In [47]:
mod2 = LogisticRegression(C=1, penalty='l1')
mod2.fit(X[~missing], y[~missing])
mod2.predict(X[missing])

array([ 1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.])

In [55]:
mod2 = LogisticRegression(C=0.4, penalty='l1')
mod2.fit(X[~missing], y[~missing])
mod2.predict(X[missing])

array([ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

Surprisingly few imputations are required to acheive reasonable estimates, with 3-10 usually sufficient. We will use 3.

In [56]:
mother_hs_imp = []

for C in 0.1, 0.4, 2:
    
    mod = LogisticRegression(C=C, penalty='l1')
    mod.fit(X[~missing], y[~missing])
    imputed = mod.predict(X[missing])
    mother_hs_imp.append(imputed)

In [57]:
mother_hs_imp

[array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
 array([ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,
         1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
         1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
 array([ 1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
         0.,  1.,  1.,  

Now we can perform 3 separate analyses, using the method of our choice, each based upon a different set of imputed values. 

We will 

In [62]:
from sklearn import linear_model

coefficients = []

for imputes in mother_hs_imp:
    
    regr = linear_model.LinearRegression()
    
    X = test_scores.drop(labels=['family_inv','prev_disab'], axis=1)
    X.loc[missing, 'mother_hs'] = imputes
    y = X.pop('score')
    regr.fit(X, y)
    coefficients.append(regr.coef_)

In [64]:
coeff_labels = ['male',
                'siblings',
                'non_english',
                'age_test',
                'non_severe_hl',
                'mother_hs',
                'early_ident',
                'non_white']

coef_df = pd.DataFrame(coefficients, columns=coeff_labels)
coef_df

,male,siblings,non_english,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,-1.067003,-2.357046,-11.595782,0.464223,5.378508,6.760373,7.626037,-9.052654
1,-1.481119,-2.738123,-9.672844,0.378593,5.513913,10.464116,7.822660,-8.541135
2,-1.747683,-2.671041,-8.653736,0.389529,5.527252,9.953979,7.636700,-8.405001


In [66]:
coef_df.mean()

male            -1.431935
siblings        -2.588737
non_english     -9.974121
age_test         0.410782
non_severe_hl    5.473224
mother_hs        9.059489
early_ident      7.695133
non_white       -8.666263
dtype: float64

In [72]:
regr_complete = linear_model.LinearRegression()
X_complete = test_scores.drop(labels=['family_inv','prev_disab'], axis=1).dropna()
y_complete = X_complete.pop('score')
regr_complete.fit(X_complete, y_complete)
pd.Series(regr_complete.coef_, index=coeff_labels)

male              1.572293
siblings         -3.388291
non_english      -8.025037
age_test          0.506523
non_severe_hl     5.772677
mother_hs        10.666549
early_ident       9.942757
non_white        -9.111430
dtype: float64

In [74]:
regr_mean = linear_model.LinearRegression()
X_mean = test_scores.drop(labels=['family_inv','prev_disab'], axis=1)
X_mean = X_mean.fillna(X_mean.mean())
y_mean = X_mean.pop('score')
regr_mean.fit(X_mean, y_mean)
pd.Series(regr_mean.coef_, index=coeff_labels)

male             -1.068601
siblings         -2.297816
non_english     -10.857035
age_test          0.454713
non_severe_hl     5.302381
mother_hs        10.367914
early_ident       7.725014
non_white        -8.774047
dtype: float64

## References

1.	Schafer JL. Multiple imputation: a primer. Stat Methods Med Res. 1999;8(1):3-15. doi:10.1177/096228029900800102.
1.	van Buuren S. Flexible Imputation of Missing Data. 2012:1-326.

---

In [2]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()